In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from tensorflow.keras.utils import to_categorical

/Users/wojtek/miniforge3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
y = to_categorical(y)

#norm=np.linalg.norm(y)
#y/=norm

list(zip(X, y))

[(array([5.1, 3.5, 1.4, 0.2]), array([1., 0., 0.], dtype=float32)),
 (array([4.9, 3. , 1.4, 0.2]), array([1., 0., 0.], dtype=float32)),
 (array([4.7, 3.2, 1.3, 0.2]), array([1., 0., 0.], dtype=float32)),
 (array([4.6, 3.1, 1.5, 0.2]), array([1., 0., 0.], dtype=float32)),
 (array([5. , 3.6, 1.4, 0.2]), array([1., 0., 0.], dtype=float32)),
 (array([5.4, 3.9, 1.7, 0.4]), array([1., 0., 0.], dtype=float32)),
 (array([4.6, 3.4, 1.4, 0.3]), array([1., 0., 0.], dtype=float32)),
 (array([5. , 3.4, 1.5, 0.2]), array([1., 0., 0.], dtype=float32)),
 (array([4.4, 2.9, 1.4, 0.2]), array([1., 0., 0.], dtype=float32)),
 (array([4.9, 3.1, 1.5, 0.1]), array([1., 0., 0.], dtype=float32)),
 (array([5.4, 3.7, 1.5, 0.2]), array([1., 0., 0.], dtype=float32)),
 (array([4.8, 3.4, 1.6, 0.2]), array([1., 0., 0.], dtype=float32)),
 (array([4.8, 3. , 1.4, 0.1]), array([1., 0., 0.], dtype=float32)),
 (array([4.3, 3. , 1.1, 0.1]), array([1., 0., 0.], dtype=float32)),
 (array([5.8, 4. , 1.2, 0.2]), array([1., 0., 0.

In [22]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_der(x):
    x = sigmoid(x)
    return x * (1 - x)

class Net:
    def __init__(self, layers, alpha=0.01):
        self.num_layers = len(layers)
        self.weights = [np.random.randn(m, n) for n, m in zip(layers[:-1], layers[1:])]
        self.biases = [np.random.randn(m) for m in layers[1:]]
        self.alpha = alpha

    def predict(self, X):
        for W, b in list(zip(self.weights, self.biases)):
            X = sigmoid(np.dot(W, X) + b)
        return X

    def feed_forward(self, X):
        activations = [X]
        for W, b in list(zip(self.weights, self.biases)):
            X = sigmoid(np.dot(W, X) + b)
            activations.append(X)
        return activations, X

    def backpropagate(self, X, y):
        delta_W = [np.zeros(W.shape) for W in self.weights]
        delta_b = [np.zeros(b.shape) for b in self.biases]

        activations, y_hat = self.feed_forward(X)
        delta = (y_hat - y) * activations[-1] * (1 - activations[-1])

        delta_b[-1] = delta
        delta_W[-1] = np.dot(delta, activations[-2].T)

        for l in range(2, self.num_layers):
            delta = np.dot(self.weights[-l + 1].T, delta) * activations[-l]
            delta_b[-l] = delta
            delta_W[-l] = np.dot(delta, activations[-l - 1].T)
        return delta_W, delta_b

    def train(self, train_data, epochs, batch_size, test_data=None):
        for e in range(epochs):
            np.random.shuffle(train_data)
            for i in range(0, len(train_data), batch_size):
                batch = train_data[i: i + batch_size]

                delta_W = [np.zeros(W.shape) for W in self.weights]
                delta_b = [np.zeros(b.shape) for b in self.biases]

                for x, y in batch:
                    batch_delta_W, batch_delta_b = self.backpropagate(x, y)
                    delta_W += batch_delta_W
                    delta_b += batch_delta_b

                self.weights = [w - self.alpha / len(batch) * dW for w, dW in zip(self.weights, delta_W)]
                self.biases = [b - self.alpha / len(batch) * db for b, db in zip(self.biases, delta_b)]
        # if test_data: print(f'Epoch {e}: accuracy: {}')

    # def score(self, test_data):
    #     X, y = test_data
    #     y_hat = [np.argmax(y_) for y_ in self.predict(X)]
    #     return np.sum(y_hat == )

In [ ]:
model = Net([4, 3, 3])
print(model.weights)
# model.predict(X[0])
model.train(list(zip(X, y)), 5, 10)

[array([[-1.83481998, -0.22531547,  0.00731514, -0.10159897],
       [ 0.50383634,  0.17753931, -0.10534116,  0.54892348],
       [ 1.40806816,  0.85336245, -0.2721946 ,  1.2526889 ]]), array([[-0.51782711,  0.94394527,  0.987623  ],
       [-0.00452818,  1.31526619,  2.83412745],
       [ 0.36520076,  2.75039259,  0.78281552]])]
